In [1]:
#Importing Necessary Packages

import numpy as np
import pandas as pd
import sys
import gc
import pickle
import datetime
from itertools import product
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
test  = pd.read_csv('data/test.csv').set_index('ID')
train_set = pd.read_pickle('sales_train_final.pkl')

In [ ]:
X_train = train_set[train_set.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = train_set[train_set.date_block_num < 33]['item_cnt_month']
X_valid = train_set[train_set.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = train_set[train_set.date_block_num == 33]['item_cnt_month']
X_test = train_set[train_set.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [2]:
model = XGBRegressor(
    max_depth=10,
    n_estimators=1500,
    min_child_weight=500, 
    colsample_bytree=0.75, 
    subsample=0.75, 
    eta=0.3,    
    seed=50)

In [ ]:
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 15)

In [ ]:
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('submission.csv', index=False)